# **ETL de metadata de Google Maps**  
**Empezamos nuestro trabajo con la extracción de los datos de archivos json**  

Importamos librerias necesarias:

In [13]:
import pandas as pd
import json
import os
import re

## **Extracción de Datos:**  

Rutas necesarias para trabajar (Colocar aquí ruta de 'metadata-sitios' y 'reviews-estados'):

In [14]:
# ruta_metadata = '../../Data/google_maps/metadata-sitios/'
# Verificar que SOLO estén los JSON en dichas carpetas y sus subcrpetas sino dará error. Configurar "ver archivos ocultos" y borrar "desktop.ini" u otros archivos que no sean los JSON.
ruta_metadata = 'C:/Users/jalvarez/Desktop/Google Maps/metadata-sitios/'
rutas_reviews = 'C:/Users/jalvarez/Desktop/Google Maps/reviews-estados/'

In [15]:
rutas_reviews_estados = {
    'California': rutas_reviews + 'review-California',
    'Florida': rutas_reviews + 'review-Florida/',
    'Hawaii': rutas_reviews + 'review-Hawaii/',
    'Nevada': rutas_reviews + 'review-Nevada/',
    'New_York': rutas_reviews + 'review-New_York/',
    'Connecticut': rutas_reviews + 'review-Connecticut/',
    'New Jersey': rutas_reviews + 'review-New_Jersey/',
    'Texas': rutas_reviews + 'review-Texas/',
    'Pennsylvania': rutas_reviews + 'review-Pennsylvania/',
    'Alaska': rutas_reviews + 'review-Alaska/',
}

Recopilar IDs de GMap: Se emplea una función para recorrer los directorios de reseñas estatales, recolectando IDs únicos de Google Maps. Este paso es crucial para correlacionar las reseñas con los metadatos de restaurantes correspondientes.

Para esto se emplea la siguiente función:

In [16]:
def recopilar_ids_gmap_de_reviews(rutas_reviews_estados):
    estados_gmap_ids = {}
    for estado, ruta_estado in rutas_reviews_estados.items():
        estados_gmap_ids[estado] = set()
        # Iterando sobre cada archivo JSON
        for directorio in os.listdir(ruta_estado):
            ruta_archivo = os.path.join(ruta_estado, directorio)
            with open(ruta_archivo, 'r') as archivo:
                for linea in archivo:
                    review_data = json.loads(linea)
                    estados_gmap_ids[estado].add(review_data['gmap_id'])
                    
    return estados_gmap_ids

Filtrar Metadatos: Con los IDs de GMap recolectados, filtramos los metadatos completos para retener solo aquellas entradas que coincidan con nuestro conjunto de datos de reseñas. Esta filtración garantiza que nuestro conjunto de datos sea preciso y relevante para el análisis.  

Para esto se emplea la siguiente función:

In [18]:
def filtro_metadata(ruta_metadata, estados_gmap_ids):
    metadata_filtrada = []
    for directorio in os.listdir(ruta_metadata):
        ruta_archivo = os.path.join(ruta_metadata, directorio)
        with open(ruta_archivo, 'r') as archivo:
            for linea in archivo:
                lugar_data = json.loads(linea)
                for estado, gmap_ids in estados_gmap_ids.items():
                    if lugar_data['gmap_id'] in gmap_ids:
                        lugar_data['state'] = estado 
                        metadata_filtrada.append(lugar_data)
                        break  
                    
    return metadata_filtrada

In [19]:
# Paso 1: Recopilar IDs de GMap de las revisiones estatales con información del estado
estado_gmap_ids = recopilar_ids_gmap_de_reviews(rutas_reviews_estados)

# Paso 2: Filtrar lugares de metadatos basados en los IDs de GMap recopilados e incluir información del estado
metadata_filtrada = filtro_metadata(ruta_metadata, estado_gmap_ids)

# Paso 3: Convertir los metadatos filtrados a un DataFrame
df_metadata_filtrada = pd.DataFrame(metadata_filtrada)

In [20]:
df_metadata_filtrada.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",California,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
1,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},California,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,Matrix International Textiles,"Matrix International Textiles, 1363 S Bonnie B...",0x80c2cf163db6bc89:0x219484e2edbcfa41,None,34.015505,-118.181839,[Fabric store],3.5,6,None,"[[Thursday, 8:30AM–5:30PM], [Friday, 8:30AM–5:...",{'Accessibility': ['Wheelchair accessible entr...,California,"[0x80c2cf042a5d9561:0xd0024ad6f81f1335, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [21]:
df_metadata_filtrada["state"].unique()

array(['California', 'Texas', 'New_York', 'Pennsylvania', 'Florida',
       'Hawaii', 'Alaska', 'Nevada', 'Connecticut', 'New Jersey'],
      dtype=object)

In [22]:
df_metadata_filtrada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388610 entries, 0 to 388609
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              388608 non-null  object 
 1   address           385427 non-null  object 
 2   gmap_id           388610 non-null  object 
 3   description       55050 non-null   object 
 4   latitude          388610 non-null  float64
 5   longitude         388610 non-null  float64
 6   category          388307 non-null  object 
 7   avg_rating        388610 non-null  float64
 8   num_of_reviews    388610 non-null  int64  
 9   price             59864 non-null   object 
 10  hours             336144 non-null  object 
 11  MISC              347994 non-null  object 
 12  state             388610 non-null  object 
 13  relative_results  369842 non-null  object 
 14  url               388610 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 44.5+ MB


Filtrado por Restaurantes:

In [23]:
restaurante_comida = [
    'restaurant', 'cafe', '\\bfood\\b', 'dining', 'eatery', 'bistro', 'bakery',
    'grill', 'kitchen', 'pizzeria', 'steakhouse', 'sushi', 'tavern', 'diner'
]

exclusiones = [
    'supplier','ATM','gas station', 'school', 'bank', 'area', 'company', 'broker', 'bark', 'stool', 'dart',
    'store', 'shop', 'bar', 'lounge', 'venue', 'service', 'club', 'remodeler', 'boutique',
    'market', 'pharmacy', 'furniture', 'grocery', 'hardware', 'book', 'garden', 'home', 'office', 
    'electronics', 'clothing', 'gift', 'toy', 'jewelry', 'florist', 'repair', 'maintenance',
    'construction', 'contractor', 'installer', 'supplier', 'wholesaler', 'retailer', 'distributor', 
    'manufacturer', 'producer', 'facility', 'center', 'park', 'gallery', 'studio', 'salon', 'spa',
    'gym', 'fitness', 'health', 'wellness', 'boutique', 'event', 'entertainment', 'amusement', 
    'recreation', 'cultural', 'education', 'tutoring', 'learning', 'training', 'consultant', 
    'counseling', 'legal', 'financial', 'insurance', 'real estate', 'accommodation', 'lodging', 
    'rental', 'automotive', 'mechanic', 'pet', 'veterinary', 'storage', 'security', 'transportation',
    'delivery', 'logistics', 'utility', 'energy', 'sanitation', 'cleaning', 'waste', 'recycling',
]

patron_de_incluidos = re.compile(r'\b(?:' + '|'.join(restaurante_comida) + r')\b', re.IGNORECASE)
patron_de_excluidos = re.compile(r'\b(?:' + '|'.join(exclusiones) + r')\b', re.IGNORECASE)

In [24]:
# Verifica si alguna de las categorías en la lista coincide con las palabras clave relacionadas con alimentos
# y se asegura de que ninguna coincida con las exclusiones.

def es_servivio_de_comida(lista_categorias):

    lista_categorias = lista_categorias if isinstance(lista_categorias, list) else []
    if any(patron_de_excluidos.search(categoria) for categoria in lista_categorias):
        return False
    return any(patron_de_incluidos.search(categoria) for categoria in lista_categorias)

restaurantes_metadata = [lugar for lugar in metadata_filtrada if es_servivio_de_comida(lugar.get('category'))]

In [25]:
df_restaurantes_metadata = pd.DataFrame(restaurantes_metadata)
df_restaurantes_metadata.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",California,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,[Restaurant],4.5,18,None,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...","{'Service options': ['Outdoor seating', 'Curbs...",California,None,https://www.google.com/maps/place//data=!4m2!3...
2,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",0x8644b59b8fe872e5:0x5e638876caa84cc3,None,30.273985,-97.719563,[Restaurant],4.5,8,None,"[[Thursday, 5PM–12AM], [Friday, 5PM–12AM], [Sa...","{'Service options': ['Delivery', 'Takeout', 'D...",Texas,None,https://www.google.com/maps/place//data=!4m2!3...


In [26]:
df_restaurantes_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33626 entries, 0 to 33625
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              33626 non-null  object 
 1   address           33601 non-null  object 
 2   gmap_id           33626 non-null  object 
 3   description       8070 non-null   object 
 4   latitude          33626 non-null  float64
 5   longitude         33626 non-null  float64
 6   category          33626 non-null  object 
 7   avg_rating        33626 non-null  float64
 8   num_of_reviews    33626 non-null  int64  
 9   price             14629 non-null  object 
 10  hours             31916 non-null  object 
 11  MISC              33479 non-null  object 
 12  state             33626 non-null  object 
 13  relative_results  27967 non-null  object 
 14  url               33626 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 3.8+ MB


Verificamos categorias:

In [27]:
categorias_expandidas = df_restaurantes_metadata.explode('category')

categorias_counts = categorias_expandidas['category'].value_counts()

df_categorias_counts = categorias_counts.reset_index()
df_categorias_counts.columns = ['Category', 'Count']

df_categorias_counts.head(5)

,Category,Count
0,Restaurant,11578
1,Mexican restaurant,3420
2,Pizza restaurant,2688
3,Chinese restaurant,2488
4,Cafe,2037


## **Trasnformación de Datos:**  

Empezamos eliminando columnas que consideramos innecesarias para un análisis:

In [28]:
drops = ['description', 'relative_results', 'MISC','hours','url']
df_restaurantes = df_restaurantes_metadata.drop(drops, axis=1)

Observamos que en la columna 'price' se encuentran valores unicos: '$', '$$', '$$$$', '$$$', '₩₩', '₩', '₩₩₩', '₩₩₩₩'
Estos indican lo siguiente:

- '$': Precio bajo
- '$$': Precio medio
- '$$$': Precio alto
- '$$$$': Precio muy alto
-
- '₩': Precio bajo en won surcoreano
- '₩₩': Precio medio en won surcoreano
- '₩₩₩': Precio alto en won surcoreano
- '₩₩₩₩': Precio muy alto en won surcoreano

A estos los transformaremos en números para una manipulación más eficiente donde:

- 1 : Precio bajo
- 2 : Precio medio
- 3 : Precio alto
- 4 : Precio muy alto

In [29]:
df_restaurantes["price"].unique()

array([None, '$', '$$', '$$$$', '$$$', '₩₩', '₩', '₩₩₩', '₩₩₩₩'],
      dtype=object)

In [30]:
def precio_numerico(price):
    if pd.isnull(price):
        return 0  
    else:
        return len(price)  

df_restaurantes['price_numeric'] = df_restaurantes['price'].apply(precio_numerico)
df_restaurantes.drop('price', axis=1, inplace=True)
df_restaurantes['price_numeric'] = df_restaurantes['price_numeric'].astype(int)

In [31]:
df_restaurantes["price_numeric"].value_counts()

price_numeric
0    18997
1     8395
2     5897
3      283
4       54
Name: count, dtype: int64

Se extrae el cod. postal y la ciudad de la columna 'addres':

In [32]:
ciudad_regex = r',\s*([^,]+),\s*[A-Z]{2}\s+\d{5}'
cod_postal_regex = r'(\d{5})$'

df_restaurantes['city'] = df_restaurantes['address'].str.extract(ciudad_regex, expand=False)
df_restaurantes['postal_code'] = df_restaurantes['address'].str.extract(cod_postal_regex, expand=False)

Reordenamos las columnas a un formato adecuado:

In [33]:
orden = ['name', 'address','state', 'city', 'postal_code', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'price_numeric', 'gmap_id','category']
df_restaurantes = df_restaurantes.reindex(columns=orden)

df_restaurantes.head(3)

,name,address,state,city,postal_code,latitude,longitude,avg_rating,num_of_reviews,price_numeric,gmap_id,category
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",California,Los Angeles,90005,34.058092,-118.292130,4.4,18,0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,[Korean restaurant]
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",California,La Mirada,90638,33.916402,-118.010855,4.5,18,0,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,[Restaurant]
2,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",Texas,Austin,78702,30.273985,-97.719563,4.5,8,0,0x8644b59b8fe872e5:0x5e638876caa84cc3,[Restaurant]


Creamos un nuevo DataFrame de dummies a partir de la columna: 'category'

In [34]:
temp_df = df_restaurantes[['gmap_id', 'category']]

# Convertir listas a cadenas
df_restaurantes['category'] = df_restaurantes['category'].apply(lambda x: ', '.join(x))

# Crear variables dummy
category_dummies = df_restaurantes['category'].str.get_dummies(', ')
restaurante_category_dummies = pd.concat([df_restaurantes[['gmap_id']], category_dummies], axis=1)

In [35]:
# Vemos el resultado:
restaurante_category_dummies.head(4)

,gmap_id,Afghani restaurant,African restaurant,American restaurant,Animal shelter,Animals,Argentinian restaurant,Armenian restaurant,Art cafe,Asian fusion restaurant,...,Western restaurant,Wholesale bakery,Wi-Fi spot,Wine cellar,Winery,Wok restaurant,Yakiniku restaurant,Yakitori restaurant,Yemenite restaurant,Yucatan restaurant
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0x8644b59b8fe872e5:0x5e638876caa84cc3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0x88dae191ee505917:0x6ba3e25388d3fad4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
lista = restaurante_category_dummies.columns.to_list()
lista

['gmap_id',
 'Afghani restaurant',
 'African restaurant',
 'American restaurant',
 'Animal shelter',
 'Animals',
 'Argentinian restaurant',
 'Armenian restaurant',
 'Art cafe',
 'Asian fusion restaurant',
 'Asian restaurant',
 'Association or organization',
 'Australian restaurant',
 'Austrian restaurant',
 'Authentic Japanese restaurant',
 'Bakery',
 'Bangladeshi restaurant',
 'Banquet hall',
 'Barbecue restaurant',
 'Basque restaurant',
 'Beer hall',
 'Belgian restaurant',
 'Beverages',
 'Bingo hall',
 'Biryani restaurant',
 'Bistro',
 'Boat tour agency',
 'Bouncy castle hire',
 'Bowling alley',
 'Brasserie',
 'Brazilian restaurant',
 'Breakfast restaurant',
 'Brewery',
 'Brewpub',
 'Bridge',
 'British restaurant',
 'Brunch restaurant',
 'Buffet restaurant',
 'Burmese restaurant',
 'Burrito restaurant',
 'Cafe',
 'Cafeteria',
 'Cajun restaurant',
 'Californian restaurant',
 'Cambodian restaurant',
 'Camp',
 'Campground',
 'Canadian restaurant',
 'Canoe & kayak tour agency',
 'Cantone

In [37]:
df_restaurantes.head(3)

,name,address,state,city,postal_code,latitude,longitude,avg_rating,num_of_reviews,price_numeric,gmap_id,category
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",California,Los Angeles,90005,34.058092,-118.292130,4.4,18,0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Korean restaurant
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",California,La Mirada,90638,33.916402,-118.010855,4.5,18,0,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Restaurant
2,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",Texas,Austin,78702,30.273985,-97.719563,4.5,8,0,0x8644b59b8fe872e5:0x5e638876caa84cc3,Restaurant


**Verificamos la integridad de los datos:**

Verificamos los duplicados por una clave y eliminamos:

In [38]:
clave = ['name', 'address', 'city', 'postal_code']
duplicados = df_restaurantes.duplicated(subset=clave, keep=False)

duplicados_potenciales = df_restaurantes[duplicados]

sorted_duplicados_potenciales = duplicados_potenciales.sort_values(by='name')

print("Potenciales duplicados ordenados por nombre:")
sorted_duplicados_potenciales.head(4)

Potenciales duplicados ordenados por nombre:


,name,address,state,city,postal_code,latitude,longitude,avg_rating,num_of_reviews,price_numeric,gmap_id,category
312,'Round The Lake Kitchen,"'Round The Lake Kitchen, 12057 PA-618, Conneau...",Pennsylvania,Conneaut Lake,16316,41.630465,-80.325579,4.5,27,0,0x88324fdf1a955263:0x6b4077533d6e6061,Restaurant
74,'Round The Lake Kitchen,"'Round The Lake Kitchen, 12057 PA-618, Conneau...",Pennsylvania,Conneaut Lake,16316,41.630465,-80.325579,4.5,27,0,0x88324fdf1a955263:0x6b4077533d6e6061,Restaurant
352,1903 Taphouse & Co.,"1903 Taphouse & Co., 175 N Main St, Bishop, CA...",California,Bishop,93514,37.361898,-118.395555,4.7,8,0,0x80be3dd544115ed9:0x5d500f8046469ea3,American restaurant
114,1903 Taphouse & Co.,"1903 Taphouse & Co., 175 N Main St, Bishop, CA...",California,Bishop,93514,37.361898,-118.395555,4.7,8,0,0x80be3dd544115ed9:0x5d500f8046469ea3,American restaurant


In [39]:
# Eliminamos duplicados:
df_restaurantes.drop_duplicates(subset=clave, inplace=True)

In [40]:
# Verificamos
duplicados = df_restaurantes.duplicated(subset=clave, keep=False)

duplicados_potenciales = df_restaurantes[duplicados]

sorted_duplicados_potenciales = duplicados_potenciales.sort_values(by='name')

sorted_duplicados_potenciales.head(4)

,name,address,state,city,postal_code,latitude,longitude,avg_rating,num_of_reviews,price_numeric,gmap_id,category


**Verificamos valores faltantes**  

Tanto en todas las columnas como en la columna 'address':

In [41]:
nulos_count = df_restaurantes.isnull().sum()
print(f"Count of missing values in each column: {nulos_count}")

Count of missing values in each column: name                 0
address             25
state                0
city               184
postal_code       2273
latitude             0
longitude            0
avg_rating           0
num_of_reviews       0
price_numeric        0
gmap_id              0
category             0
dtype: int64


In [42]:
filas_con_nulos = df_restaurantes[df_restaurantes.isnull().any(axis=1)]
print("Filas con al menos un valor nulo:")
filas_con_nulos.head(3)

Filas con al menos un valor nulo:


,name,address,state,city,postal_code,latitude,longitude,avg_rating,num_of_reviews,price_numeric,gmap_id,category
40,李小龍台吃,"〒11362 New York, Queens, Northern Blvd, 李小龍台吃",New_York,NaN,NaN,40.770059,-73.735522,4.0,27,0,0x89c289efdb82221b:0xed627c2af97c2069,Restaurant
89,glazed,"Texas, Sugar Land, glazed邮政编码: 77479",Texas,NaN,77479,29.590894,-95.625716,4.3,8,0,0x8640e6bd6e861357:0xb96bb36e5cd02ee3,Restaurant
1088,Me Bakery,"New York, Flushing, 47th Ave, Me Bakery邮政编码: 1...",New_York,NaN,11358,40.752362,-73.785966,4.5,24,0,0x89c261a1df3470c3:0x1e414e9bf91c369f,Cafe


Establecimientos con nulos en la columna 'address':

In [43]:
address_nulo = df_restaurantes[df_restaurantes['address'].isnull()]
address_nulo.head(3)

,name,address,state,city,postal_code,latitude,longitude,avg_rating,num_of_reviews,price_numeric,gmap_id,category
4482,Pasquale's Pizzeria and Restaurant at Wallkill,None,New_York,None,None,41.710932,-74.112969,4.1,54,0,0x89dd29ab5dd170f1:0x7cb3c8e119a5bbe9,Pizza restaurant
5636,Namaste Tashi Delek,None,New_York,None,None,40.759291,-73.884518,4.3,95,0,0x89c25f072d742ce9:0x2a930399a15215e3,"Nepalese restaurant, Halal restaurant"
17850,Sabaidee Thai Grille,None,California,None,None,37.275179,-119.273197,4.0,38,0,0x809ac6a6aa8b8cdf:0xeb219547c732cc3c,Thai restaurant


Buscando en google maps o en la web manualmente las direcciones (latitud y longitud) de estos establecimientos sin datos en la columna 'address',  
entendimos que estos son establecimientos cerrados o nunca existieron en primer lugar. Por estas razones seran eliminados:

In [44]:
df_restaurantes = df_restaurantes.dropna(subset=['address'])

Verificamos nuevamente los nulos del dataframe:

In [45]:
df_restaurantes.isna().sum()

name                 0
address              0
state                0
city               159
postal_code       2248
latitude             0
longitude            0
avg_rating           0
num_of_reviews       0
price_numeric        0
gmap_id              0
category             0
dtype: int64

Por practicidad cambiaremos los nulos en 'SD' (Sin Dato), para la columna 'city' y para la columna 'postal_code': 0000 (Lo mismo que Sin Dato pero en código postal)

In [46]:
df_restaurantes["city"] = df_restaurantes["city"].fillna("SD")

df_restaurantes["postal_code"] = df_restaurantes["postal_code"].fillna("0000")

In [47]:
# Verificamos:
df_restaurantes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33356 entries, 0 to 33625
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            33356 non-null  object 
 1   address         33356 non-null  object 
 2   state           33356 non-null  object 
 3   city            33356 non-null  object 
 4   postal_code     33356 non-null  object 
 5   latitude        33356 non-null  float64
 6   longitude       33356 non-null  float64
 7   avg_rating      33356 non-null  float64
 8   num_of_reviews  33356 non-null  int64  
 9   price_numeric   33356 non-null  int32  
 10  gmap_id         33356 non-null  object 
 11  category        33356 non-null  object 
dtypes: float64(3), int32(1), int64(1), object(7)
memory usage: 3.2+ MB


Observamos los tipos de datos de las columnas, vemo que 'postal_code' es tipo object. Lo cambiaremos a int:

In [48]:
df_restaurantes["postal_code"] = df_restaurantes["postal_code"].astype(int)

## **Carga de Datos:**  

Guardaremos nuestros dataframes en archivos parquet:

Datos de los restaurantes, Dataframe 'df_restaurantes'. Lo usaremos para filtrar información de las reviews principalmente y luego para análisis.

In [49]:
df_restaurantes.to_parquet("../../Data/data_procesada/restaurantes.parquet")

Datos dummies de las categorias de los restaurantes, Dataframe 'restaurante_category_dummies'. Lo usaremos para los modelos de ML.

In [50]:
restaurante_category_dummies.to_parquet("../../Data/data_procesada/dummies_restaurantes.parquet")